In [457]:
import plotly as ply
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import numpy as np

import sklearn.linear_model as lm
from sklearn.model_selection import train_test_split

from haversine import haversine, Unit, inverse_haversine

In [458]:
def find_cl_angle(ax, ay, bx, by, direction):
    a = np.array([ax, ay])
    b = np.array([bx, by])
    
    if direction == 'n':
        cx = bx
        cy = by + 0.05
        if ax >= bx:
            sign = -1
        else:
            sign = 1
    if direction == 'e':
        cx = bx + 0.05
        cy = by
        if ay >= by:
            sign = 1
        else:
            sign = -1
    if direction == 's':
        cx = bx
        cy = by - 0.05
        if ax >= bx:
            sign = 1
        else:
            sign = -1
    if direction == 'w':
        cx = bx - 0.05
        cy = by
        if ay >= by:
            sign = -1
        else:
            sign = 1
    
    c = np.array([cx, cy])

    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return angle * sign

# Load Camera Metadata

In [459]:
df_cameras = pd.read_csv('data/cam_meta_rev2.csv')
df_cameras = df_cameras.loc[df_cameras['c_lat'] > 0]
df_cameras['cl_angle'] = df_cameras.apply(lambda x: find_cl_angle( x['c_long'], x['c_lat'], x['cam_long'], x['cam_lat'], x['dir']), axis=1)

In [460]:
df_cameras.head()

,camera_id,camera_name,hpweren_camera_description,cam_lat,cam_long,station,dir,c_lat,c_long,elevation,certainty,cl_angle
1,hpwren1_north,bm-n-mobo-c,Big Black Mountain,33.159927,-116.808092,bm,n,33.173626,-116.811270,4055.0,NaN,0.227974
2,hpwren1_east,bm-e-mobo-c,Big Black Mountain,33.159927,-116.808092,bm,e,33.158781,-116.790230,4055.0,NaN,-0.064085
3,hpwren1_south,bm-s-mobo-c,Big Black Mountain,33.159927,-116.808092,bm,s,33.157932,-116.807962,4055.0,NaN,0.065022
4,hpwren1_west,bm-w-mobo-c,Big Black Mountain,33.159927,-116.808092,bm,w,33.159091,-116.858706,4055.0,NaN,0.016519
5,hpwren2_north,bl-n-mobo-c,Black Mountain,32.981417,-117.116480,bl,n,32.981788,-117.116520,1600.0,NaN,0.108221


# Load Landmarks

In [461]:
df_lm = pd.read_csv('data/landmarks_processed.csv')
df_lm = df_lm.rename(columns={'lat': 'lm_lat', 'long': 'lm_long'})

In [462]:
df_lm.head()

,landmark,station,dir,lm_lat,lm_long,x_pixel,y_pixel,x_res,y_res,intersection,distance
0,white building - southwest,om,s,32.593892,-116.845919,2900,1715,3072,2048,0,0.150093
1,white building - southwest,om,w,32.593892,-116.845919,423,1685,3072,2048,0,0.150093
2,white building - south,om,s,32.594298,-116.844774,1780,1930,3072,2048,0,0.052190
3,communication tower building - east,om,n,32.595487,-116.844141,2545,1725,3072,2048,0,0.095800
4,large bush,om,e,32.599053,-116.839254,318,1420,3072,2048,0,0.698102


# Combine Camera Metadata & Landmarks

In [463]:
def find_angle(ax, ay, bx, by, cx, cy):
    a = np.array([ax, ay])
    b = np.array([bx, by])
    c = np.array([cx, cy])

    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return angle

In [464]:
df_merged = df_cameras.merge(df_lm, left_on=['station', 'dir'], right_on=['station', 'dir'], how='inner')
df_merged = df_merged[['station', 'dir', 'hpweren_camera_description', 'landmark', 'cam_lat', 'cam_long', 'c_lat', 'c_long', 'lm_lat', 'lm_long', 'x_pixel', 'y_pixel', 'x_res', 'y_res', 'elevation', 'distance', 'intersection', 'cl_angle']]
df_merged['angle'] = df_merged.apply(lambda x: find_angle(x['c_long'], x['c_lat'], x['cam_long'], x['cam_lat'], x['lm_long'], x['lm_lat']), axis=1)

In [465]:
df_merged.head()

,station,dir,hpweren_camera_description,landmark,cam_lat,cam_long,c_lat,c_long,lm_lat,lm_long,x_pixel,y_pixel,x_res,y_res,elevation,distance,intersection,cl_angle,angle
0,bm,n,Big Black Mountain,sharp turn in trail,33.159927,-116.808092,33.173626,-116.811270,33.168644,-116.797208,2808,1414,3072,2048,4055.0,1.402096,0,0.227974,1.123460
1,bm,e,Big Black Mountain,point in lake Sutherland,33.159927,-116.808092,33.158781,-116.790230,33.116483,-116.776150,3018,1230,3072,2048,4055.0,5.672844,0,-0.064085,0.872729
2,bm,e,Big Black Mountain,sharp turn in trail,33.159927,-116.808092,33.158781,-116.790230,33.168644,-116.797208,321,1410,3072,2048,4055.0,1.402096,0,-0.064085,0.739396
3,bm,s,Big Black Mountain,nucal egg ranch,33.159927,-116.808092,33.157932,-116.807962,33.072523,-116.776892,1067,1127,3072,2048,4055.0,10.144026,1,0.065022,0.277835
4,bm,s,Big Black Mountain,lake southerland - channel opening,33.159927,-116.808092,33.157932,-116.807962,33.105561,-116.780867,900,1200,3072,2048,4055.0,6.555269,0,0.065022,0.399234


# Convert to Training Dataset

In [466]:
import sklearn.linear_model as lin_model
import sklearn.model_selection as model
import sklearn.preprocessing as preprop
from sklearn.metrics import r2_score

In [467]:
def find_ratio(pix, res):
    return abs(pix-(res/2))/(res/2)

df_merged['x_ratio'] = df_merged.apply(lambda x: find_ratio(x['x_pixel'], x['x_res']), axis=1)
df_merged['y_ratio'] = df_merged.apply(lambda x: find_ratio(x['y_pixel'], x['y_res']), axis=1)

In [468]:
X_test = df_merged[['x_ratio', 'y_ratio', 'elevation']].loc[df_merged['intersection'] == 1]
X_test['elevation'] = X_test['elevation'] - np.median(X_test['elevation'])
y_test = df_merged['angle'].loc[df_merged['intersection'] == 1]

X_train = df_merged[['x_ratio', 'y_ratio', 'elevation']].loc[df_merged['intersection'] == 0]
X_train['elevation'] = X_train['elevation'] - np.median(X_train['elevation'])
y_train = df_merged['angle'].loc[df_merged['intersection'] == 0]

In [469]:
scaler = preprop.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)

In [470]:
#X_train, X_test, y_train, y_test = model.train_test_split(X, y, test_size=0.2, random_state=6)

In [471]:
clf = lin_model.Ridge(alpha=11.5, max_iter=1000)#, selection='random')
clf.fit(X_train, y_train)

Ridge(alpha=11.5, max_iter=1000)

In [472]:
print(clf.coef_)
y_pred = clf.predict(X_test)

[ 0.24079129 -0.00132079 -0.01295714]


In [473]:
np.array(y_test)

array([0.27783537, 0.23212931, 0.61770058, 0.22521155, 0.12764786,
       0.11674425, 0.38375344, 0.61714282, 0.10711167, 0.73511634])

In [474]:
print(r2_score(y_test, y_pred))

0.8536878066204144


# TRY WITHOUT LOW RES

# Triangulation

In [475]:
df_intersections = df_merged.loc[df_merged['intersection'] == 1].reset_index(drop=True)

In [476]:
test_lms = set(df_intersections['landmark'])

In [477]:
test_lms

{'Palomar Observatory right',
 'lake henshaw delta',
 'nucal egg ranch',
 'otay resevoir - north tip',
 'target - san clemente'}

In [478]:
df_intersections

,station,dir,hpweren_camera_description,landmark,cam_lat,cam_long,c_lat,c_long,lm_lat,lm_long,...,y_pixel,x_res,y_res,elevation,distance,intersection,cl_angle,angle,x_ratio,y_ratio
0,bm,s,Big Black Mountain,nucal egg ranch,33.159927,-116.808092,33.157932,-116.807962,33.072523,-116.776892,...,1127,3072,2048,4055.0,10.144026,1,0.065022,0.277835,0.305339,0.100586
1,bh,e,Boucher Hill,Palomar Observatory right,33.334628,-116.919328,33.334624,-116.918709,33.348388,-116.859694,...,1011,3072,2048,5500.0,5.747043,1,-0.005346,0.232129,0.320312,0.012695
2,hp,w,High Point,Palomar Observatory right,33.363018,-116.836220,33.363096,-116.837515,33.348388,-116.859694,...,693,2048,1536,6180.0,2.720259,1,-0.060387,0.617701,0.842773,0.097656
3,mg,n,Mesa Grande,lake henshaw delta,33.188858,-116.760659,33.221119,-116.761198,33.266743,-116.744179,...,1078,3072,2048,4280.0,8.795076,1,0.016690,0.225212,0.194661,0.052734
4,mg,s,Mesa Grande,nucal egg ranch,33.188858,-116.760659,33.167663,-116.760892,33.072523,-116.776892,...,1117,3072,2048,4280.0,13.023885,1,-0.010993,0.127648,0.126953,0.090820
5,sm,s,Mt. San Miguel,otay resevoir - north tip,32.696885,-116.936262,32.613700,-116.933834,32.628680,-116.926238,...,1250,3072,2048,2550.0,7.641891,1,0.029180,0.116744,0.082031,0.220703
6,om,w,Otay Mountain,otay resevoir - north tip,32.594762,-116.844694,32.594836,-116.851777,32.628680,-116.926238,...,1153,3072,2048,3565.0,8.518187,1,-0.010424,0.383753,0.507161,0.125977
7,sclm,n,San Clemente,target - san clemente,33.431419,-117.597341,33.478431,-117.579006,33.464032,-117.605505,...,1183,3072,2048,870.0,3.704763,1,-0.371860,0.617143,0.582031,0.155273
8,sjh,s,San Juan Hills,target - san clemente,33.502942,-117.603118,33.497394,-117.602863,33.464032,-117.605505,...,1152,3072,2048,1177.0,4.332222,1,0.045826,0.107112,0.094401,0.125000
9,vo,n,Santa Ysabel,lake henshaw delta,33.138264,-116.617095,33.161819,-116.618152,33.266743,-116.744179,...,1147,3072,2048,5500.0,18.544640,1,0.044822,0.735116,0.609375,0.120117


In [479]:
def find_slope(angle, cl_angle, direction, xpix, xres):
    if direction == 'n':
        if xpix < (xres/2):
            return np.tan((np.pi/2)+angle+cl_angle)
        else:
            return np.tan((np.pi/2)-angle+cl_angle)
    if direction == 'e':
        if xpix < (xres/2):
            return np.tan(angle+cl_angle)
        else:
            return np.tan(-angle+cl_angle)
    if direction == 's':
        if xpix < (xres/2):
            return np.tan((3*np.pi/2)+angle+cl_angle)
        else:
            return np.tan((3*np.pi/2)-angle+cl_angle)
    if direction == 'w':
        if xpix < (xres/2):
            return np.tan(np.pi+angle+cl_angle)
        else:
            return np.tan(np.pi-angle+cl_angle)

# Triangulations

In [480]:
for lm in test_lms:
    
    data1 = df_intersections.loc[df_intersections['landmark'] == lm].iloc[0]
    data2 = df_intersections.loc[df_intersections['landmark'] == lm].iloc[1]

    data_scaled1 = scaler.transform([data1[['x_ratio', 'y_ratio', 'elevation']]])
    data_scaled2 = scaler.transform([data2[['x_ratio', 'y_ratio', 'elevation']]])

    ang1 = clf.predict(data_scaled1)
    ang2 = clf.predict(data_scaled2)

    m1 = find_slope(ang1, data1['cl_angle'], data1['dir'], data1['x_pixel'], data1['x_res'])
    b1 = data1['cam_lat'] - (m1 * data1['cam_long'])
    m2 = find_slope(ang2, data1['cl_angle'], data2['dir'], data2['x_pixel'], data2['x_res'])
    b2 = data2['cam_lat'] - (m2 * data2['cam_long'])
    
    x_coord = (b2 - b1)/(m1-m2)
    y_coord = (m1 * x_coord) + b1
    dist_acc = haversine((data1['lm_long'], data1['lm_lat']), (x_coord, y_coord), unit=Unit.MILES)
    print(lm)
    print("X: {}, Y: {}".format(x_coord, y_coord))
    print("Distance From Actual: ", dist_acc)
    print("\n")

otay resevoir - north tip
X: [-116.92981981], Y: [32.63312074]
Distance From Actual:  0.2838201082455849


nucal egg ranch
X: [-116.76598597], Y: [33.03685955]
Distance From Actual:  1.3415314399794458


lake henshaw delta
X: [-116.73884179], Y: [33.33782538]
Distance From Actual:  2.2404861190671816


target - san clemente
X: [-117.6079411], Y: [33.49334556]
Distance From Actual:  0.9535160819846488


Palomar Observatory right
X: [-116.84580125], Y: [33.3543773]
Distance From Actual:  0.9779528480808412




In [539]:
df_merged.columns

Index(['station', 'dir', 'hpweren_camera_description', 'landmark', 'cam_lat',
       'cam_long', 'c_lat', 'c_long', 'lm_lat', 'lm_long', 'x_pixel',
       'y_pixel', 'x_res', 'y_res', 'elevation', 'distance', 'intersection',
       'cl_angle', 'angle', 'x_ratio', 'y_ratio'],
      dtype='object')

# Single Camera Model

In [532]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import random

In [533]:
train_ind = random.sample(set(np.linspace(0, len(df_merged)-1, len(df_merged)).astype(int)), 100)
test_ind = list(set(np.linspace(0, len(df_merged)-1, len(df_merged)).astype(int)) - set(train_ind))

In [540]:
X_train = df_merged[['x_ratio', 'y_ratio', 'elevation']].iloc[train_ind]
#y_train = df_merged[['angle', 'distance']].iloc[train_ind]
X_train['elevation'] = X_train['elevation'] - np.median(X_train['elevation'])
y_train = df_merged[['distance']].iloc[train_ind]

X_test = df_merged[['x_ratio', 'y_ratio', 'elevation']].iloc[test_ind]
#y_test = df_merged[['angle', 'distance']].iloc[test_ind]
X_test['elevation'] = X_test['elevation'] - np.median(X_test['elevation'])
y_test = df_merged[['distance']].iloc[test_ind]

scaler = preprop.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [541]:
#applying polynomial regression degree 2
poly = PolynomialFeatures(degree=2, include_bias=True)
x_train_trans = poly.fit_transform(X_train)
x_test_trans = poly.transform(X_test)
#include bias parameter
#lr = LinearRegression()
lr = lin_model.Ridge(alpha=[0.3], max_iter=1000)#, selection='random')
lr.fit(x_train_trans, y_train)
y_pred = lr.predict(x_test_trans)
print(r2_score(y_test, y_pred))

0.15085203290673332


In [542]:
#applying polynomial regression degree 2
poly = PolynomialFeatures(degree=2, include_bias=True)
x_train_trans = poly.fit_transform(X_train)
x_test_trans = poly.transform(X_test)
#include bias parameter
#lr = LinearRegression()
lr = lin_model.ElasticNet(alpha=.0001, max_iter=1000)#, selection='random')
lr.fit(x_train_trans, y_train)
y_pred = lr.predict(x_test_trans)
print(r2_score(y_test, y_pred))

0.15136474739534256


In [543]:
lr = lin_model.Lasso(alpha=0.0001, max_iter=1000)#, selection='random')
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(r2_score(y_test, y_pred))

0.12789692826246246


In [524]:
print(lr.coef_)

[-2.33628216  0.36553373]


In [489]:
def adjust_angle(angle, cl_angle, direction, xpix, xres):
    if direction == 'n':
        if xpix < (xres/2):
            return (np.pi/2)+angle+cl_angle
        else:
            return (np.pi/2)-angle+cl_angle
    if direction == 'e':
        if xpix < (xres/2):
            return angle+cl_angle
        else:
            return -angle+cl_angle
    if direction == 's':
        if xpix < (xres/2):
            return (3*np.pi/2)+angle+cl_angle
        else:
            return (3*np.pi/2)-angle+cl_angle
    if direction == 'w':
        if xpix < (xres/2):
            return np.pi+angle+cl_angle
        else:
            return np.pi-angle+cl_angle

In [490]:
for i, ind in enumerate(test_ind):
    ang_adj = adjust_angle(y_pred[i][0], df_merged['cl_angle'].iloc[ind], df_merged['dir'].iloc[ind], df_merged['x_pixel'].iloc[ind], df_merged['x_res'].iloc[ind])
    ang_adj = ang_adj *180/np.pi
    new_coord = inverse_haversine((df_merged[['cam_lat', 'cam_long']].iloc[ind]), y_pred[i][1], ang_adj, unit=Unit.MILES)
    
    dist_acc = haversine((data1['lm_lat'], data1['lm_long']), (new_coord[0], new_coord[1]), unit=Unit.MILES)
    
    print(ind)
    print("X: {}, Y: {}".format(new_coord[1], new_coord[0]))
    print("Distance From Actual: ", dist_acc)
    print("\n")

IndexError: invalid index to scalar variable.

In [553]:
X_new = np.linspace(-3, 3, 200).reshape(200, 1)
X_new_poly = poly.transform(X_new)
y_new = lr.predict(X_new_poly)
#plt.plot(X_new, y_new, "r-", linewidth=2, label="Predictions")
plt.plot(X_train, y_train, "b.",label='Training points')
plt.plot(X_test, y_test, "g.",label='Testing points')
plt.xlabel("X")
plt.ylabel("y")
plt.legend()
plt.show()

ValueError: X shape does not match training shape

In [343]:
clf2 = lin_model.Lasso(alpha=.05, max_iter=1000)#, selection='random')
clf2.fit(X_train, y_train)

print(clf2.coef_)
y_pred = clf2.predict(X_test)

[-8.76634327]


In [344]:
print(r2_score(y_test, y_pred))

0.20501484162745542
